In [490]:
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from torch.utils.data import ConcatDataset, Subset, DataLoader
from featurization.data_utils import load_data_from_smiles, construct_loader
import pandas as pd
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.metrics.functional import average_precision, auroc
import pytorch_lightning as pl
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import Adam
import torch.nn as nn

In [8]:
chembl_4_smiles = pd.read_csv('/home/dionizije/Documents/DAO/data/chembl_4_smiles.csv')[['smiles', 'withdrawn']]

In [66]:
chembl_4_smiles = chembl_4_smiles.sample(frac=1, random_state=0)

In [35]:
X, y = load_data_from_smiles(chembl_4_smiles['smiles'], chembl_4_smiles['withdrawn'])

RDKit ERROR: [09:49:53] UFFTYPER: Unrecognized charge state for atom: 14
RDKit ERROR: [09:49:53] UFFTYPER: Unrecognized charge state for atom: 14
RDKit ERROR: [09:49:57] UFFTYPER: Unrecognized atom type: Ca (0)
RDKit ERROR: [09:49:57] UFFTYPER: Unrecognized atom type: Ca (30)
RDKit ERROR: [09:49:57] UFFTYPER: Unrecognized charge state for atom: 21
RDKit ERROR: [09:49:57] UFFTYPER: Unrecognized charge state for atom: 21
RDKit ERROR: [09:49:59] UFFTYPER: Unrecognized charge state for atom: 5
RDKit ERROR: [09:49:59] UFFTYPER: Unrecognized charge state for atom: 5
RDKit ERROR: [09:50:02] UFFTYPER: Unrecognized charge state for atom: 12
RDKit ERROR: [09:50:02] UFFTYPER: Unrecognized charge state for atom: 12
RDKit ERROR: [10:03:18] UFFTYPER: Unrecognized charge state for atom: 8
RDKit ERROR: [10:03:18] UFFTYPER: Unrecognized charge state for atom: 8
RDKit ERROR: [10:03:31] UFFTYPER: Unrecognized charge state for atom: 8
RDKit ERROR: [10:03:31] UFFTYPER: Unrecognized charge state for atom: 8

In [294]:
data_loader = construct_loader(X, y, batch_size=12)

In [491]:
train_test_splitter = StratifiedKFold(n_splits=5, random_state=0)
train_val_splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=0)

/home/dionizije/anaconda3/envs/withdrawn/lib/python3.6/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [492]:
for k, (train_index, test_index) in enumerate(train_test_splitter.split(data_loader.dataset, y)):
    test_data_loader = DataLoader(Subset(data_loader.dataset, test_index.tolist()))
    test_data_labels = [i.y[0] for i in test_data_loader.dataset]
    train_data_loader = DataLoader(Subset(data_loader.dataset, train_index.tolist()))
    train_labels = [i.y[0] for i in train_data_loader.dataset]
    
    for train_index, val_index in train_val_splitter.split(train_data_loader.dataset, train_labels):
        train_data_loader = DataLoader(Subset(train_data_loader.dataset, train_index.tolist()))
        val_data_loader = DataLoader(Subset(train_data_loader.dataset, val_index.tolist()))